# Using data analysis to find the best Wordle starting word
In chess, part of what makes good players good is that they've memorized openings. Because every chess game starts in the same position and the combinatorial explosion of possible positions doesn't happen until a few moves in, it's feasible to memorize the optimal starting few moves, regardless of what the opponent plays.

If this is something that exists in chess, then would it be possible to do something similar in Wordle, to find an optimal or near-optimal starting move?

tl;dr: use **"soare"** as your starting word. Read on to learn more about why.

## About Wordle

[Wordle](https://www.powerlanguage.co.uk/wordle/) is a word game where the objective is to guess a 5-letter word. The word to guess is different each day, and you have 6 guesses to get it right. Once you try guessing a word, the game marks each letter you entered in one of three possible colors: Green if the letter is in the solution word in the correct place, Yellow if the letter is in the solution word but in the incorrect place, and gray if it's not in the solution word.

Here are the official instructions as mentioned in the game website:

![Official instructions indicating the meaning of green, yellow, and gray tiles](img/official_instructions.png)

## A detail that was left out from the instructions

Consider these attempts:

![Game attempts that shows word Alaap has the first two A's gray and last one green. The word Araba shows with the first A yellow and the other two gray](img/yellow_green_mechanism_cropped.png)

Even though we know that the letter A is part of the answer, not all A's in the attempts are colored, which seems to contradict the instructions, where it said that gray letters aren't part of the solution word.

As it turns out, the colors can also tell us the **number of instances** of a specific letter in a solution. In the example above, the solution only has one A, so Wordle makes sure to only color as green or yellow a single instance of the letter A in any given guess.

## In search of the best starting word
Like in chess, each game of Wordle starts with the same starting position. At the start, you have no information about the word (other its length, which is always 5). This means that there must be an optimal first word that would, on average, maximize one's chance of guessing the word within the six allowed attempts, and that one could use every time. Everyone has their starting word or words that they have chosen based on anecdotal success or other heuristics, but I wanted to find a word that is objectively good according to a data-driven metric.

## But first, we need a word list
Wordle has its own dictionary of allowed words, and it doesn't accept any attempts of words that aren't in that list. The best starting word we're looking for needs to be within that dictionary or it won't be useful. One approach here would be to use another word list, such as the Scrabble dictionary, or some other 1-gram word corpus online. Then, after finding the best word, we could try this word in Wordle to ensure it was valid. But wouldn't it be better if we could simply have the list of words that Wordle checks against for validity? Maybe looking at the source code for the game will reveal the list...

So I did Right Click -> Inspect Element and that's basically all it took to find the list.

Under the Network tab in the Developer Tools, after refreshing the page, a file appears called something like "main.e65ce0a5.js" that contains the main code for the game. Under the Preview sub-tab, I searched for some five letter words, and lo and behold, there was the list.

What was surprising, though, was that there were actually two word lists in the code right next to each other:

![The source code showing two arrays of five-letter words](img/two_word_arrays.png)

The code is obfuscated, so it's not possible to see what the variable names were originally. Searching for recent Wordle solutions such as "tangy" or "abbey", it's clear that past solutions are in the first array but not in the second. So it turns out that the first array, which has 2315 entries, contains solutions. The second array, with 10657 words, has all non-solution words that are valid guesses, sorted in alphabetical order.

> Sidenote: Solutions are on the first array _in order_, so one only needs to keep on going along the list and one will find all past and future solutions. Now, there's no fun in just knowing the solution and winning in one move. I mean, if you wanted to do that, you could just look up the solution for the day online. Instead, let's keep on searching for the single best starting word.

I grabbed the words from each array and transferred them to text files words_solutions.txt and words_other.txt respectively, with one word per line. Then I read them into a Python script:

In [1]:
with open("words_solutions.txt", "r") as file:
    words_solutions = {line.strip() for line in file}
with open("words_other.txt", "r") as file:
    words_other = {line.strip() for line in file}
words_all = words_solutions | words_other
print("len(words_solutions):\t", len(words_solutions))
print("len(words_other):\t", len(words_other))
print("len(words_all):\t\t", len(words_all))

len(words_solutions):	 2315
len(words_other):	 10657
len(words_all):		 12972


## Defining some helper functions

In [2]:
### Helpers ###

def avg(_list):
    return sum(_list)/len(_list)

# computes the average number of reduced words when using this word as a guess.
def compute_score(guess_word, metric):
    return avg([metric(guess_word, actual_word) for actual_word in list(words_solutions)])

# Finds the word that maximizes the given metric. 
def find_optimal_word(metric, words_to_check=words_all):
    max_score = -1
    max_score_word = None
    for guess_word in words_to_check:
        score = compute_score(guess_word, metric)
        if score > max_score:
            max_score = score
            max_score_word = guess_word
    return max_score_word, max_score

## Defining some metrics that measure how good a guess is
We define six metrics:
1. `metric_num_green_letters` counts the number of green tiles.
1. `metric_letter_overlap` counts the number of distinct letters shared between the guess word and the solution word, regardless of their count.
1. `metric_num_yellow_green` counts the number of tiles that are either yellow or green.
1. `metric_hybrid` is a sum of `metric_num_green_letters` and `metric_num_yellow_green`. It's like `metric_num_yellow_green`, but it assigns 2 points to green tiles and 1 to yellow tiles, instead of equal score per tile.
1. `metric_num_reduced_words` measures how many words out of all possible solutions are ruled out when playing this guess, based on the yellow/green tiles revealed.
1. `metric_percent_reduced_word_pool` is exactly the same as `metric_num_reduced_words`, but represented as a percent of all possible solutions so that it's easier to interpret.

The last two are much slower to compute than the rest, but are a more accurate representation of what we're looking for when searching for the best word.

In [3]:
### Metrics ###

def metric_num_green_letters(guess_word, actual_word):
    return sum([x==y for x, y in zip(guess_word, actual_word)])

def metric_letter_overlap(guess_word, actual_word):
    return len(set(guess_word) & set(actual_word))

def metric_num_yellow_green(guess_word, actual_word):
    guess = list(guess_word)
    actual = list(actual_word)
    guess.sort()
    actual.sort()
    i = 0
    j = 0
    result = 0
    while i < len(guess) and j < len(actual):
        if guess[i] == actual[j]:
            i += 1
            j += 1
            result += 1
            continue
        if guess[i] < actual[j]:
            i += 1
            continue
        if guess[i] > actual[j]:
            j += 1
            continue
    return result

def metric_hybrid(guess_word, actual_word):
    return metric_num_green_letters(guess_word, actual_word) + metric_num_yellow_green(guess_word, actual_word)

# After guessing a word, the game colors letters indicating restrictions on the final words.
# This metric computes, on average, how many words are eliminated from the pool of possible solutions due to not meeting those restrictions.
# The bigger this metric is, the smaller the word pool for the second guess, which would in theory translate to an easier guess.
def metric_num_reduced_words(guess_word, actual_word):
    if guess_word == actual_word:
        return len(words_solutions) - 1
    
    is_green = [l == r for l, r in zip(guess_word, actual_word)]

    guess_letters = set(guess_word)

    count_requirements = {}
    for letter in guess_letters:
        guess_count = sum([x == letter for x in guess_word])
        actual_count = sum([x == letter for x in actual_word])

        green_count = sum([x == y for x, y in zip(guess_word, actual_word)])
        yellow_count = min(actual_count - green_count, guess_count - green_count)
        gray_count = guess_count - green_count - yellow_count

        count_requirements[letter] = {
            'min': green_count + yellow_count,
            'max': (green_count + yellow_count) if gray_count > 0 else 5
        }
    
    reduced_word_count = 0

    for word in words_solutions:
        # word is guess or solution. Don't filter out these words.
        if word == guess_word or word == actual_word:
            continue

        # anything that isn't green indicates that the letter won't be in that spot in the solution, so make a filter for that.
        if any((green != (letter == guess_letter) for green, letter, guess_letter in zip(is_green, word, guess_word))):
            # green letter not in word. Or gray/yellow letter in word. Filter out.
            reduced_word_count += 1
            continue

        break_early = False
        for letter in set(word):
            if break_early:
                break
            if letter in count_requirements:
                letter_count = sum([letter == x for x in word])
                counts_map = count_requirements[letter]
                if letter_count < counts_map['min'] or letter_count > counts_map['max']:
                    # min/max conditions not met. Filter out.
                    reduced_word_count += 1
                    break_early = True
                    break
        if break_early:
            continue
    
    return reduced_word_count

# This metric is similar to num_reduced_words, but it's given as a percentage to make it easier to interpret.
def metric_percent_reduced_word_pool(guess_word, actual_word):
    return 100 * metric_num_reduced_words(guess_word, actual_word) / (len(words_solutions) - 1)


## Finding the optimal word for each metric
After having defined the metrics, we compute which out of all allowed words maximizes the metric. Note that these metric numbers can't be compared to each other, since it would be comparing "apples to oranges".

In [4]:
find_optimal_word(metric_num_yellow_green)

('orate', 1.7892008639308856)

In [5]:
find_optimal_word(metric_num_green_letters)

('saree', 0.6803455723542117)

In [6]:
find_optimal_word(metric_letter_overlap)

('orate', 1.7892008639308856)

In [7]:
find_optimal_word(metric_hybrid)

('soare', 2.428077753779698)

## Checking these words with the most reliable metric
The metric `metric_percent_reduced_word_pool`, which mentions on average what percent of the remaining word pool of possible answers becomes eliminated after playing a certain word, is prohibitively expensive to compute for all words. It can, however, be computed for a handful of words in a reasonable amount of time.

This metric is in my opinion the most accurate of the ones mentioned in this blog post, since instead of using rough heuristics, it's getting at the actual thing that we want the starting word to do: reduce the number of remaining word possibilities the most.

Below is the metric as computed for the metric-optimal words mentioned above. In this case, it *is* possible to compare the different numbers shown, since they're computed with the same metric.

In [8]:
compute_score('orate', metric_percent_reduced_word_pool)

88.49661465285006

In [9]:
compute_score('saree', metric_percent_reduced_word_pool)

88.77083990584131

In [10]:
compute_score('soare', metric_percent_reduced_word_pool)

90.55655219146912

## The best starting word

Based on the scores above, the winner for best starting words is **"soare"**! with a whopping 90.6% average reduction in the word pool size.

It should be noted that it's very likely that there's a better word out there with a better score on the `percent_reduced_word_pool` metric, but because the metric is expensive to compute, it's not feasible to try all words on it, and it's necessary to start with heuristic metrics and then check that metric to see how it does, as was done in the above analysis.

If you want to check how your favorite starting word compares to "soare" or the other metric-optimizing words seen above, simply run `compute_score('<your word here>', metric_percent_reduced_word_pool)` and you'll be able to see how well it scores.

## Epilogue: Trying out other words

In [11]:
compute_score('house', metric_percent_reduced_word_pool)

84.86302737959096

In [12]:
compute_score('earth', metric_percent_reduced_word_pool)

82.86370314229674

In [13]:
compute_score('ouija', metric_percent_reduced_word_pool)

77.78441302915432